PyAnimeList by Patrick Tjahjadi

Program to retrieve Anime/Donghua data from MyAnimeList, including score, year, genre, etc.

Allows users to filter Anime/Donghua based on these attributes with a sorting feature.

Search for your favourite Anime/Donghua or simply look for recommendation with the filtering and sorting feature!

In [1]:
# Imported Libraries
from jikanpy import Jikan
import pandas as pd
import time
from IPython.display import clear_output
import dill

In [2]:
# Set up data for anime from 2000 to 2020 for retrieval using the Jikan API

jikan = Jikan()

years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
         2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
seasons = ['winter', 'spring', 'summer', 'fall']

myanimelist = []

In [ ]:
# Retrieve anime data through Jikan
# Time delay of 7 seconds per year for API rate limiting
for year in years:
    for season in seasons:
        myanimelist.append(jikan.season(year = year, season = season))
    time.sleep(7)

In [2]:
# Store the Python data into bite streams for faster future processing
# dill.dump_session('my_anime_list.db')

# We can load this later instead of retrieving data again
dill.load_session('my_anime_list.db')


In [3]:
# Collect all necessary attributes: Title, Score, Members, Genre, Producers, Year, Season and Synopsis
animedata = []
for animeseason in myanimelist:
    for show in animeseason['anime']:
        animedata.append([show['title'], show['score'], show['members'], ', '.join(genre['name'] for genre in show['genres']), 
                        ', '.join(producer['name'] for producer in show['producers']), animeseason["season_year"],
                        animeseason["season_name"], show['synopsis']])
        

In [4]:
# Create a dataframe to store Anime data and remove duplicate entries
anime_df = pd.DataFrame(animedata, columns = ["Title", "Score", "Members", "Genre", "Producers", "Year", "Season", "Synopsis",])
anime_df.index.name = "Anime ID"
anime_df.drop_duplicates(subset="Title", keep = 'first', inplace = True)

In [10]:
# Function to retrieve anime based on filtering and sorting input
def get_my_anime(output_anime_df):
    list_of_queries = []
    list_of_sort = ["None"]
    while (1):
        print("Your queries: \n"+", ".join(list_of_queries))
        method = input("Search anime based on (Title, Score, Members, Genre, Year, Season or Synopsis)? Otherwise, input 0.\n")
        if (method == "0"):
            break
        elif (method.lower() in ["title", "score", "members", "genre", "year", "season", "synopsis"]):
            output_anime_df = query_my_anime(output_anime_df, method, list_of_queries)
    output_anime_df = sort_my_anime(output_anime_df, list_of_queries, list_of_sort)
    clear_output(wait=True)
    print("Your queries: \n"+", ".join(list_of_queries))
    print("Your sorting method: \n"+list_of_sort[0])
    return output_anime_df

# Function to filter anime based on attributes
def query_my_anime(interim_df, method, list_of_queries):
    if (method.lower() in ["title", "genre", "producers", "season", "synopsis"]):
        query_content = input("Search by anime "+method.capitalize()+":\n")
        interim_df = interim_df.query('{}.str.contains("{}")'.format(method.capitalize(), query_content), engine = 'python')
        list_of_queries.append("{}: {}".format(method.capitalize(), query_content))
    elif (method.lower() in ["score", "members", "year"]):
        operator = input("Find anime "+method.capitalize()+" less than, equal to, greater than, or range (L = Less, E = Equal, G = Greater, R = Range)?\n")
        if (operator.lower() in ["g", "greater", "greater than"]):
            value = input("Greater than which "+method.capitalize()+ "?\n")
            interim_df = interim_df.query('{} > {}'.format(method.capitalize(), value))
            list_of_queries.append("{} > {}".format(method.capitalize(), value))
        elif (operator.lower() in ["e", "equal", "equal to"]):
            value = input("Equal to which "+method.capitalize()+ "?\n")
            interim_df = interim_df.query('{} == {}'.format(method.capitalize(), value))
            list_of_queries.append("{} = {}".format(method.capitalize(), value))
        elif (operator.lower() in ["l", "less", "less than"]):
            value = input("Less than which "+method.capitalize()+ "?\n")
            interim_df = interim_df.query('{} < {}'.format(method.capitalize(), value))
            list_of_queries.append("{} < {}".format(method.capitalize(), value))
        elif (operator.lower() in ["r", "range"]):
            value_low = input("Between which values inclusive? Set lower limit:\n")
            value_high = input("Between which values inclusive? Set upper limit:\n")
            interim_df = interim_df.query('{} > {} and {} < {}'.format(method.capitalize(), value_low, method.capitalize(), value_high))
            list_of_queries.append("{} <= {} <= {}".format(value_low, method.capitalize(), value_high))
    clear_output(wait=True)        
    return interim_df

# Functions to sort the order of anime to be output
def sort_my_anime(interim_df, list_of_queries, list_of_sort):
    clear_output(wait = True)
    print("Your queries: \n"+", ".join(list_of_queries))
    while (1):
        sort_attribute = input("Any sorting method (Title, Score, Members, Genre, Year, Season or Synopsis)? Otherwise, input 0.\n")
        if (sort_attribute.lower() in ["title", "score", "members", "genre", "year", "season", "synopsis"]):
            while (1):
                sort_method = input("Ascending or Descending (A = Ascending, D = Descending)?")
                if (sort_method.lower() in ["a", "ascending"]):
                    interim_df = interim_df.sort_values(sort_attribute.capitalize(), ascending = True)
                    list_of_sort[0] = sort_attribute.capitalize()+": Ascending"
                    return interim_df
                elif (sort_method.lower() in ["d", "descending"]):
                    interim_df = interim_df.sort_values(sort_attribute.capitalize(), ascending = False)
                    list_of_sort[0] = sort_attribute.capitalize()+": Descending"
                    return interim_df
        elif (sort_attribute == "0"):
            return interim_df

In [11]:
query_df = get_my_anime(anime_df)
query_df

Your queries: 

Your sorting method: 
Title: Descending


,Title,Score,Members,Genre,Producers,Year,Season,Synopsis
Anime ID,,,,,,,,
12683,ēlDLIVE,6.18,40735,"Action, Sci-Fi, Space, Police, Shounen",Studio Pierrot,2017,Winter,Chuuta Kokonose is an orphan who lives with hi...
5440,xxxHOLiC Shunmuki,8.09,57169,"Mystery, Comedy, Psychological, Supernatural, ...",Production I.G,2009,Winter,"For the appropriate price, your dearest wish c..."
6387,xxxHOLiC Rou,8.23,55379,"Mystery, Supernatural",Production I.G,2010,Spring,10 years after the events of xxxHOLiC Shunmuki...
3033,xxxHOLiC Movie: Manatsu no Yoru no Yume,7.97,54765,"Mystery, Comedy, Psychological, Supernatural, ...",Production I.G,2005,Summer,"Summer break has arrived, but while his other ..."
4841,xxxHOLiC Kei,8.25,111552,"Mystery, Comedy, Psychological, Supernatural, ...",Production I.G,2008,Spring,Kimihiro Watanuki's life was never really norm...
3410,xxxHOLiC,8.02,238114,"Comedy, Drama, Mystery, Psychological, Superna...",Production I.G,2006,Spring,Kimihiro Watanuki can see spirits and other as...
7829,s.CRY.ed Alteration II: Quan,6.83,3526,"Action, Sci-Fi, Adventure, Super Power, Shounen",Sunrise,2012,Winter,Second part of the s.CRY.ed recap.
7611,s.CRY.ed Alteration I: Tao,6.92,4418,"Action, Sci-Fi, Adventure, Super Power, Shounen",Sunrise,2011,Fall,Special recap of s.CRY.ed to commemorate the 1...
649,s.CRY.ed,7.38,70643,"Action, Sci-Fi, Adventure, Super Power, Drama",Sunrise,2001,Summer,A strange environmental phenomenon 22 years ag...
